In [23]:
import os
import pandas as pd
import numpy as np

HOME_PATH = os.path.expanduser('~')
PICKLES_PATH = HOME_PATH + '/Projects/tminm/pickles/'
HEAVY_PICKLES_PATH = HOME_PATH + '/Projects/tminm/heavy_pickles/'

In [24]:
vocab_pickle = pd.read_pickle(HEAVY_PICKLES_PATH + 'vocab.pickle')
vocab = vocab_pickle.tolist()

In [25]:
K = 10
V = len(vocab)
# xi and D give a notion in relicating the 'b and e' corpus.
xi = 200
D = 6327

alpha = np.random.rand(K)
beta = np.zeros((K, V))

for r, row in enumerate(beta):
    beta[r] = np.random.dirichlet(np.full(V, 1./V))

In [26]:
def mean_parameter(arr):
    return np.exp(arr) / np.sum(np.exp(arr))

In [29]:
theta = np.random.dirichlet(alpha)
generated_corpus = {}
for i in range(D):
    generated_corpus[i] = {}
    N = np.random.poisson(xi)
    for j in range(N):
        topic_distrib = np.random.multinomial(1, theta)
        z = np.where(topic_distrib == 1)[0][0]
        word_distrib = np.random.multinomial(1, beta[z])
        w = np.where(word_distrib == 1)[0][0]
        w_key = vocab[w]
        if w_key not in generated_corpus[i]:
            generated_corpus[i][w_key] = 0
        generated_corpus[i][w_key] += 1
        
corpus_panda = pd.Series(generated_corpus)
corpus_panda.to_pickle(HEAVY_PICKLES_PATH + 'gen_corpus.pickle')

[ 0.13337925  0.00157562  0.16691397  0.16805026  0.15109834  0.01744935
  0.16507376  0.00155755  0.1485754   0.04632649]


ValueError: sum(pvals[:-1]) > 1.0

In [28]:
seq_generated_corpus = {}
for i in range(D):
    seq_generated_corpus[i] = {}
    N = np.random.poisson(xi)
    eta = np.random.normal(alpha)
    for j in range(N):
        mp_eta = mean_parameter(eta)
        topic_distrib = np.random.multinomial(1, mp_eta)
        z = np.where(topic_distrib == 1)[0][0]
        mp_beta = mean_parameter(beta[z])
        word_distrib = np.random.multinomial(1, mp_beta)
        w = np.where(word_distrib == 1)[0][0]
        w_key = vocab[w]
        if w_key not in seq_generated_corpus[i]:
            seq_generated_corpus[i][w_key] = 0
        seq_generated_corpus[i][w_key] += 1
    beta = np.random.normal(beta)
    alpha = np.random.normal(alpha)
    
seq_corpus_panda = pd.Series(seq_generated_corpus)
seq_corpus_panda.to_pickle(HEAVY_PICKLES_PATH + 'seq_gen_corpus.pickle')